Connect to database

In [2]:
import datajoint as dj

dj.config['database.host'] = "gl-ash.biostr.washington.edu"
dj.config['database.user'] = "gabby"
dj.config['database.port'] = 3306

dj.conn()

/home/gl-willow/mambaforge/envs/gabby_spyglass_env2/lib/python3.10/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-18 16:38:55,139][INFO]: DataJoint 0.14.6 connected to gabby@gl-ash.biostr.washington.edu:3306


DataJoint connection (connected) gabby@gl-ash.biostr.washington.edu:3306

Imports

In [33]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import spyglass.common as sgc
import spyglass.lfp as lfp
import spyglass.position as sgp
from spyglass.lfp.analysis.v1 import LFPBandV1

from ripple_detection.core import (
    gaussian_smooth,
    get_envelope,
)

import figpack.views as vv
import figpack_experimental.views as vve

from gl_spyglass.utils.common_neural_functions import validate_references, apply_referencing
from gl_spyglass.custom_spyglass_tables.grouped_ripple import LFPBandGroup, RippleTimesGroup

In [4]:
os.environ['FIGPACK_BUCKET'] = 'gillespielab'

# TODO: update this with your own custom FIGPACK_API_KEY (contact Jeremy Magland if you don't have one)
os.environ['FIGPACK_API_KEY'] = '17979e3bc8880ea91520c1dc2e8ab6a97fc665893fe2728e9d642923c3dcf31c'

### Choose subj, date, epoch to look at

In [17]:
subj = 'pippin'
date = 20210421
epoch = 1

nwb_file_name = f'{subj}{date}_.nwb'
interval_list_name = (
    sgc.TaskEpoch() & {"nwb_file_name": nwb_file_name, "epoch": epoch}
).fetch1("interval_list_name")
pos_interval_list_name = (
    sgc.IntervalList()
    & {"nwb_file_name": nwb_file_name, "pipeline": "position"}
).fetch("interval_list_name")[epoch - 1]


### Load in ripple-filtered dataframe and ripple times

#### Set LFP + ripple parameters

In [14]:
ref_on = True
lfp_electrode_group_name = 'good_single_elecs'
lfp_sampling_rate = 1_000
lfp_filter_name = 'LFP 0-400 Hz'
artifact_params_name = 'mad_7_0.66_thresh_200ms'
ripple_filter_name = 'Ripple 100-250 Hz'
ripple_param_name = 'shvartsman_sd4_part2'

#### Load in LFP

In [18]:
# get validated references
electrodes_df, _ = validate_references(nwb_file_name, is_copy=True)

# narrow down electrodes_df to good electrodes and only one per tetrode
electrodes_df = electrodes_df[electrodes_df['bad_channel'] == 'False']
electrodes_df = pd.DataFrame(
    [
        electrodes_df[electrodes_df['electrode_group_name'] == i].iloc[0]
        for i in np.unique(electrodes_df['electrode_group_name'].values)
    ]
)

good_elecs_df = electrodes_df[
    (electrodes_df['bad_channel'] == 'False')
]
good_single_elecs_df = pd.DataFrame(
    [
        good_elecs_df[good_elecs_df["electrode_group_name"] == i].iloc[0]
        for i in np.unique(good_elecs_df["electrode_group_name"].values)
    ]
)
good_single_elecs = good_single_elecs_df['electrode_id'].values
lfp_electrode_group_name = 'good_single_elecs'

lfp_sampling_rate = 1_000

lfp_filter_name = 'LFP 0-400 Hz'
lfp_s_key = {
    'nwb_file_name': nwb_file_name,
    'lfp_electrode_group_name': lfp_electrode_group_name,
    'target_interval_list_name': interval_list_name,
    'filter_name': lfp_filter_name,
    'filter_sampling_rate': 30_000,  # sampling rate of the data (Hz)
    'target_sampling_rate': lfp_sampling_rate,  # sampling rate of the lfp output (Hz)
}

lfp_merge_id = (lfp.LFPOutput.LFPV1() & lfp_s_key).fetch1('merge_id')
lfp_key = {
    'merge_id': lfp_merge_id,
}
lfp_df = (lfp.LFPOutput & lfp_key).fetch1_dataframe()
lfp_df.columns = good_single_elecs  # rename lfp columns to their original elec ids

# apply referencing if referencing is on
if ref_on:
    lfp_df = apply_referencing(lfp_df, electrodes_df)

In [19]:
# 0 the index to the start of the interval
int_start_time, int_end_time = (sgc.IntervalList() & {'nwb_file_name': nwb_file_name, 'interval_list_name': interval_list_name}).fetch1('valid_times')[0]
lfp_df.index = lfp_df.index - int_start_time

#### Load in ripple times

In [21]:
# select ripples to include
trodes_pos_params_name = 'default'
pos_s_key = {
    "nwb_file_name": nwb_file_name,
    "interval_list_name": pos_interval_list_name,
    "trodes_pos_params_name": trodes_pos_params_name,
}
pos_key = (sgp.v1.TrodesPosSelection() & pos_s_key).fetch1("KEY")
pos_merge_key = (sgp.PositionOutput.merge_get_part(pos_key)).fetch1("KEY")
pos_merge_id = pos_merge_key['merge_id']
lfp_artifact_s_key = {
    'nwb_file_name': nwb_file_name,
    'lfp_electrode_group_name': lfp_electrode_group_name,
    'target_interval_list_name': interval_list_name,
    'filter_name': lfp_filter_name,
    'filter_sampling_rate': 30_000,  # I'm pretty sure this is the sampling rate for the original data but not sure
    'artifact_params_name': artifact_params_name,
}
artifact_removed_interval_list_name = (lfp.v1.LFPArtifactRemovedIntervalList() & lfp_artifact_s_key).fetch1('artifact_removed_interval_list_name')
rip_interval_list_name = artifact_removed_interval_list_name
lfp_band_s_key = {
    'lfp_merge_id': lfp_merge_id,
    'filter_name': ripple_filter_name,
    'filter_sampling_rate': lfp_sampling_rate,
    'target_interval_list_name': rip_interval_list_name,
    'lfp_band_sampling_rate': lfp_sampling_rate,
    'nwb_file_name': lfp.LFPOutput.merge_get_parent({"merge_id": lfp_merge_id}).fetch1("nwb_file_name"),
}
lfp_band_key = (LFPBandV1 & lfp_band_s_key).fetch1("KEY")

# Load in ripple data from the ripple group table
lfp_band_key['ripple_param_name'] = ripple_param_name
ripple_data = (RippleTimesGroup().RippleTimes() & lfp_band_key).fetch1_dataframe()
ripple_times = ripple_data[['start_time', 'end_time']].values

In [22]:
# 0 the index to the start of the interval
ripple_times = ripple_times - int_start_time

#### Load in the zscored envelope

In [24]:
detector = 'shvartsman'
ripple_band_df = (LFPBandV1() & lfp_band_key).fetch1_dataframe()
env = get_envelope(ripple_band_df)
env = gaussian_smooth(env, sigma=0.004, sampling_frequency=1000)
if detector == 'shvartsman':
    baselines, deviations = (LFPBandGroup() & {'band_group_name': nwb_file_name, 'band_group_filter_name': 'Ripple 100-250 Hz'}).fetch('elec_baselines', 'elec_deviations', limit=1)
    baselines = baselines[0]
    deviations = deviations[0]
    zscore_env = (env - baselines) / deviations
else:
    from scipy.stats import zscore
    zscore_env = zscore(env, nan_policy="omit")
zscore_env_df = pd.DataFrame(zscore_env, index=ripple_band_df.index)

# update columns to match the electrode id names like in lfp_df
ca1_and_ref_elec_ids = electrodes_df.loc[(electrodes_df['region_name'] == 'ca1'), 'electrode_id'].values
zscore_env_df.columns = ca1_and_ref_elec_ids

# 0 the index to the start of the interval
zscore_env_df.index = zscore_env_df.index - int_start_time

### Separate the LFP into intervals using ripple times

In [29]:
# optimized by chatgpt
n_timestamps_ms = 500
n_timestamps_s = 0.5
n_ripples = len(ripple_times)

ca1_elec_ids = electrodes_df.loc[(electrodes_df['region_name'] == 'ca1'), 'electrode_id'].values

# Preallocate arrays
lfp_interval_data = np.full((n_ripples, n_timestamps_ms, len(ca1_elec_ids)), np.nan, dtype='float16')
win_starts = np.zeros(n_ripples)
rip_starts = np.zeros(n_ripples)
rip_ends = np.zeros(n_ripples)

# --- Step 1: Compute all window intervals first ---
win_starts_list = []
win_ends_list = []

for rip_start, rip_end in ripple_times:
    rip_duration = rip_end - rip_start
    if rip_duration < n_timestamps_s:
        win_buffer = (n_timestamps_s - rip_duration) / 2
        win_start = rip_start - win_buffer
        win_end = rip_end + win_buffer
    else:
        rip_center = (rip_end + rip_start) / 2
        win_start = rip_center - n_timestamps_s / 2
        win_end = rip_center + n_timestamps_s / 2

    win_starts_list.append(win_start)
    win_ends_list.append(win_end)

win_starts = np.array(win_starts_list)
win_ends = np.array(win_ends_list)
rip_starts = np.array([r[0] for r in ripple_times])
rip_ends = np.array([r[1] for r in ripple_times])

# --- Step 2: Restrict lfp_df to relevant time range ---
global_start = min(win_starts)
global_end = max(win_ends)
lfp_subset = lfp_df.loc[(lfp_df.index >= global_start) & (lfp_df.index < global_end), ca1_elec_ids]

lfp_times = lfp_subset.index.values
lfp_values = lfp_subset.values.astype('float16')

# --- Step 3: Extract each ripple's data using precomputed intervals ---
for r in tqdm(range(n_ripples)):
    win_start = win_starts[r]
    win_end = win_ends_list[r]

    # Boolean mask for this interval
    mask = (lfp_times >= win_start) & (lfp_times < win_end)
    rip_data = lfp_values[mask, :]

    if len(rip_data) == (n_timestamps_ms - 1):
        rip_data = np.vstack([rip_data, np.full((1, len(ca1_elec_ids)), np.nan, dtype='float16')])
    elif len(rip_data) < (n_timestamps_ms - 1):
        print(f'skipping ripple {r}')
        continue

    lfp_interval_data[r, :, :] = rip_data

  0%|          | 0/1588 [00:00<?, ?it/s]

### Plot LFP intervals

In [30]:
rip_lfp_ints = vve.MultiChannelIntervals(
    sampling_frequency_hz = 1000,
    data=lfp_interval_data,
    window_start_times_sec=win_starts,
    interval_start_times_sec=rip_starts,
    interval_end_times_sec=rip_ends,
)

### Separate the zscored envelope into intervals using ripple times

In [31]:
# Preallocate arrays
env_interval_data = np.full((n_ripples, n_timestamps_ms, len(ca1_elec_ids)), np.nan, dtype='float16')
win_starts = np.zeros(n_ripples)
rip_starts = np.zeros(n_ripples)
rip_ends = np.zeros(n_ripples)

# --- Step 1: Compute all window intervals first ---
win_starts_list = []
win_ends_list = []

for rip_start, rip_end in ripple_times:
    rip_duration = rip_end - rip_start
    if rip_duration < n_timestamps_s:
        win_buffer = (n_timestamps_s - rip_duration) / 2
        win_start = rip_start - win_buffer
        win_end = rip_end + win_buffer
    else:
        rip_center = (rip_end + rip_start) / 2
        win_start = rip_center - n_timestamps_s / 2
        win_end = rip_center + n_timestamps_s / 2

    win_starts_list.append(win_start)
    win_ends_list.append(win_end)

win_starts = np.array(win_starts_list)
win_ends = np.array(win_ends_list)
rip_starts = np.array([r[0] for r in ripple_times])
rip_ends = np.array([r[1] for r in ripple_times])

# --- Step 2: Restrict zscore_env_df to relevant time range ---
global_start = min(win_starts)
global_end = max(win_ends)
zscore_env_subset = zscore_env_df.loc[(zscore_env_df.index >= global_start) & (zscore_env_df.index < global_end), ca1_elec_ids]

zscore_env_times = zscore_env_subset.index.values
zscore_env_values = zscore_env_subset.values.astype('float16')

# --- Step 3: Extract each ripple's data using precomputed intervals ---
for r in tqdm(range(n_ripples)):
    win_start = win_starts[r]
    win_end = win_ends_list[r]

    # Boolean mask for this interval
    mask = (zscore_env_times >= win_start) & (zscore_env_times < win_end)
    rip_data = zscore_env_values[mask, :]

    if len(rip_data) == (n_timestamps_ms - 1):
        rip_data = np.vstack([rip_data, np.full((1, len(ca1_elec_ids)), np.nan, dtype='float16')])
    elif len(rip_data) < (n_timestamps_ms - 1):
        print(f'skipping ripple {r}')
        continue

    env_interval_data[r, :, :] = rip_data


  0%|          | 0/1588 [00:00<?, ?it/s]

### Plot zscored envelope intervals

In [32]:
rip_zscore_env_ints = vve.MultiChannelIntervals(
    sampling_frequency_hz = 1000,
    data=env_interval_data,
    window_start_times_sec=win_starts,
    interval_start_times_sec=rip_starts,
    interval_end_times_sec=rip_ends,
)

### Gather and display the views

In [35]:
layout = vv.Box(
    title=f"{nwb_file_name} {interval_list_name} {'(referenced)' if ref_on else '(not referenced)'}",
    direction='horizontal',
    items=[
        vv.LayoutItem(rip_lfp_ints, title='ripple intervals in referenced lfp', stretch=0.5),
        vv.LayoutItem(rip_zscore_env_ints, title='ripple intervals in zscored ripple power envelopes', stretch=0.5),
    ]
)
layout.show(title=f"{nwb_file_name} {interval_list_name} {'referenced' if ref_on else ''} detected ripples - shvartsman", upload=True)

Found 8 files to upload, total size: 90.10 MB
Uploading 8 files in batches of 20 with up to 16 concurrent uploads per batch...
Processing batch 1/1 (8 files)...
Uploaded 1/8: index.html
Uploaded 2/8: assets/index-GPjx4QpG.css
Uploaded 3/8: extension-figpack-experimental.js
Uploaded 4/8: extension_manifest.json
Uploaded 5/8: assets/neurosift-logo-CLsuwLMO.png
Uploaded 6/8: assets/index-BY1Hwjm4.js
Uploaded 7/8: data.zarr/.zmetadata
Uploaded 8/8: data.zarr/_consolidated_0.dat
Creating manifest...
Total size: 90.10 MB
Uploading manifest.json...
Finalizing figure...
Upload completed successfully
View the figure at: https://figures.figpack.org/figures/default/6cc3f9d5027c9dcd8e06ed4e/index.html


'https://figures.figpack.org/figures/default/6cc3f9d5027c9dcd8e06ed4e/index.html'

A couple of notes for the figpack itself:
- to proceed to the next ripple, toggle the right left arrows at the bottom left
- the plus / minus allows you to increase or decrease the separation between the electrodes, try playing around with it to see which is the best for seeing what you want to see

Also, while this tutorial focuses this view on ripples, it can be used to visual any set of intervals you want!